# Decentralized Machine Learning

In [61]:
import warnings
warnings.simplefilter('ignore')
import logging
import pickle
import shutil
import time

import numpy as np
import tensorflow as tf

from eth_utils import is_address
from web3.auto import w3
from web3 import Web3, HTTPProvider
# warnings.filterwarnings('ignore')

### Client Setup

In [105]:
from web3 import Web3, HTTPProvider


web3 = Web3(HTTPProvider('http://localhost:8545'))
# web3 = Web3(web3.currentProvider)

PASSPHRASE = 'panda'
TEST_ACCOUNT = web3.eth.coinbase

acct = web3.personal.newAccount(PASSPHRASE)
print("New Account:", acct)
print("Balance:", web3.eth.getBalance(acct))
web3.eth.sendTransaction({"from": TEST_ACCOUNT, "to": acct, "value": 9999999999})
print("Balance:", web3.eth.getBalance(acct))
# print("Accounts on the Network:", web3.eth.accounts)
# print("Personal Accounts:", web3.personal.listAccounts)

New Account: 0x4A8625EE042BE02e52133f7b816e97C6db8FdA80
Balance: 0
Balance: 9999999999


### Contract Deployment

In [108]:
from solc import compile_source
import pprint


# CONTRACT_NAME = 'Greeter.sol'
CONTRACT_NAME = '../contracts/Query.sol'
ARG1 = 3
ARG2 = [0, 1, 2]

def compile_source_file(file_path):
   with open(file_path, 'r') as f:
      source = f.read()

   return compile_source(source)


def deploy_contract(w3, contract_interface, arg1, arg2):
    tx_hash = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']).constructor(arg1, arg2).transact({"from": acct})

    address = w3.eth.getTransactionReceipt(tx_hash)['contractAddress']
    return address


def wait_for_receipt(w3, tx_hash, poll_interval):
   while True:
       tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
       if tx_receipt:
         return tx_receipt
       time.sleep(poll_interval)
    
contract_source_path = CONTRACT_NAME
compiled_sol = compile_source_file(contract_source_path)

pp = pprint.PrettyPrinter()

contract_id, contract_interface = compiled_sol.popitem()
# pp.pprint(contract_interface['abi'])

# print(web3.eth.accounts)
# print(acct)
web3.personal.unlockAccount(acct, PASSPHRASE)

address = deploy_contract(web3, contract_interface, ARG1, ARG2)
print("Deployed {0} to: {1}\n".format(contract_id.replace('<stdin>:', ''), address))
pp.pprint(contract_interface['abi'])

Deployed Query to: 0x03F642FceB93212d5710c6B5f8B0344150Ae47E1

[{'constant': False,
  'inputs': [],
  'name': 'inverseScale',
  'outputs': [{'name': '', 'type': 'bool'}],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': True,
  'inputs': [{'name': '', 'type': 'uint256'}],
  'name': 'keyList',
  'outputs': [{'name': '', 'type': 'int256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'vectorLength',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'totalNumData',
  'outputs': [{'name': '', 'type': 'int256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': False,
  'inputs': [{'name': 'update', 'type': 'int256[]'},
             {'name': 'key', 'type': 'int256'},
             {'name': 'numData', 'type': 'int256'}],
  'nam

### Event Listening

In [109]:
contractAbi = [{'constant': False,
  'inputs': [],
  'name': 'inverseScale',
  'outputs': [{'name': '', 'type': 'bool'}],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': True,
  'inputs': [{'name': '', 'type': 'uint256'}],
  'name': 'keyList',
  'outputs': [{'name': '', 'type': 'int256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'vectorLength',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'totalNumData',
  'outputs': [{'name': '', 'type': 'int256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': False,
  'inputs': [{'name': 'update', 'type': 'int256[]'},
             {'name': 'key', 'type': 'int256'},
             {'name': 'numData', 'type': 'int256'}],
  'name': 'sendResponse',
  'outputs': [{'name': '', 'type': 'int256[]'}],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': True,
  'inputs': [{'name': '', 'type': 'int256'}, {'name': '', 'type': 'uint256'}],
  'name': 'weights',
  'outputs': [{'name': '', 'type': 'int256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'numberOfResponses',
  'outputs': [{'name': '', 'type': 'int256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'moreThanOne',
  'outputs': [{'name': '', 'type': 'bool'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'name': '_vectorLength', 'type': 'uint256'},
             {'name': '_keyList', 'type': 'int256[]'}],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'constructor'},
 {'anonymous': False,
  'inputs': [{'indexed': False, 'name': 'client', 'type': 'address'}],
  'name': 'ClientSelected',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': False, 'name': 'n', 'type': 'int256'}],
  'name': 'ResponseReceived',
  'type': 'event'}]

In [111]:
def txn_digest(txn):
    return "Contract address: {0}\nEvent: {1}\nArg: {2}".format(
        txn['address'], txn['event'], txn['args'])

def event_callback(arg):
    print(txn_digest(arg))

contract_obj = web3.eth.contract(
   address=address,
   abi=contractAbi)

# pp.pprint(contract_obj.events)

# tx_retval = contract_obj.functions.sendResponse([0, 1, 2], 1, 3).call({'from': TEST_ACCOUNT})
tx_hash = contract_obj.functions.sendResponse([0, 1, 2], 0, 3).transact({'from': TEST_ACCOUNT})
tx_receipt = web3.eth.getTransactionReceipt(tx_hash)
print(tx_receipt)


# client_selected_event.createFilter({'filter': {'arg1':10}})

# contract_obj.functions.keyList().call({'from': TEST_ACCOUNT})

log1 = contract_obj.events.ResponseReceived().processReceipt(tx_receipt)
# log2 = contract_obj.events.ResponseReceived().processReceipt(tx_receipt)

pp.pprint(log1[0])
event_callback(log1[0])

# contract_obj.functions.sendResponse([0, 1, 2], 1, 3).transact({'from': TEST_ACCOUNT})
# contract_obj.functions.sendResponse([0, 1, 2], 2, 3).transact({'from': TEST_ACCOUNT})
# contract_obj.functions.inverseScale().transact({'from': TEST_ACCOUNT})

AttributeDict({'transactionHash': HexBytes('0xba697ecbd7ef4230c8c7c280635107084f82aa4d82cf8b194352c426d8515740'), 'transactionIndex': 0, 'blockHash': HexBytes('0xffe705f8bf83cb0d92ceea49d2d31176ed9b3e9d4f114017ccc0f4a14137e207'), 'blockNumber': 6, 'gasUsed': 50955, 'cumulativeGasUsed': 50955, 'contractAddress': None, 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0xba697ecbd7ef4230c8c7c280635107084f82aa4d82cf8b194352c426d8515740'), 'blockHash': HexBytes('0xffe705f8bf83cb0d92ceea49d2d31176ed9b3e9d4f114017ccc0f4a14137e207'), 'blockNumber': 6, 'address': '0x03F642FceB93212d5710c6B5f8B0344150Ae47E1', 'data': '0x0000000000000000000000000000000000000000000000000000000000000002', 'topics': [HexBytes('0xe28285942d518e21b1c49e5621e987261f87683241d76e22fdb0e498ac853242')], 'type': 'mined'})], 'status': 1, 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [103]:
event_filter = contract_obj.eventFilter('ResponseReceived', {'fromBlock': 'latest'})
print(event_filter.get_all_entries())
my_filter = web3.eth.filter({"address": address})
print(my_filter.get_all_entries())
new_filter = contract_obj.events.ResponseReceived().createFilter(fromBlock = 'latest')
print(new_filter.get_all_entries())
block_filter = web3.eth.filter('latest')
print(block_filter.get_all_entries())

/usr/local/lib/python3.6/site-packages/web3/utils/decorators.py:14: DeprecationWarning: eventFilter is deprecated in favor of contract.events.<event name>.createFilter
  return self.method(obj, *args, **kwargs)


[]
[]
[]


ConnectionError: HTTPConnectionPool(host='localhost', port=8545): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x116179ba8>: Failed to establish a new connection: [Errno 61] Connection refused',))